In [1]:
import numpy as np
import pandas as pd
import random
from glob import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd
import math
# visualization
import cv2
from glob import glob
# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix, roc_curve
import timm
# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torch.optim.swa_utils import AveragedModel, SWALR
from transformers import get_cosine_schedule_with_warmup
from collections import defaultdict
# import matplotlib.pyplot as plt
# Albumentations for augmentations
import albumentations as A
import albumentations
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    seed = 1
    model_name = "tf_efficientnetv2_b2"
    train_bs = 16
    valid_bs = train_bs*4
    image_size = 1024
    epochs = 25
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(CFG.device)

cuda


In [3]:
df = pd.read_csv("train_5folds.csv")
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,fold
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,1
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,1
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,1
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,1
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,0


In [4]:
is_hol = df['cancer'] == 1
df_try = df[is_hol]
df1 = df.append([df_try]*3,ignore_index=True)
print(len(df1))

58180


In [5]:
def init_logger(log_file='train1.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()
now = datetime.now()
datetime_now = now.strftime("%m/%d/%Y, %H:%M:%S")
LOGGER.info(f"Date :{datetime_now}")

Date :02/24/2023, 14:16:56


In [6]:
from albumentations import DualTransform
image_size = 1024
def isotropically_resize_image(img, size, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC):
    h, w = img.shape[:2]
    if max(w, h) == size:
        return img
    if w > h:
        scale = size / w
        h = h * scale
        w = size
    else:
        scale = size / h
        w = w * scale
        h = size
    interpolation = interpolation_up if scale > 1 else interpolation_down
    resized = cv2.resize(img, (int(w), int(h)), interpolation=interpolation)
    return resized


class IsotropicResize(DualTransform):
    def __init__(self, max_side, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC,
                 always_apply=False, p=1):
        super(IsotropicResize, self).__init__(always_apply, p)
        self.max_side = max_side
        self.interpolation_down = interpolation_down
        self.interpolation_up = interpolation_up

    def apply(self, img, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC, **params):
        return isotropically_resize_image(img, size=self.max_side, interpolation_down=interpolation_down,
                                          interpolation_up=interpolation_up)

    def apply_to_mask(self, img, **params):
        return self.apply(img, interpolation_down=cv2.INTER_NEAREST, interpolation_up=cv2.INTER_NEAREST, **params)

    def get_transform_init_args_names(self):
        return ("max_side", "interpolation_down", "interpolation_up")
    
data_transforms = {
    "train": A.Compose([
        # A.Resize(image_size, image_size),
        # IsotropicResize(max_side = image_size),
        # A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
        # A.RandomBrightnessContrast(),
        A.VerticalFlip(p=0.5),   
        A.ColorJitter(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Cutout(max_h_size=int(image_size*0.4), max_w_size=int(image_size*0.4), num_holes=1, p=0.5), 
        # A.OneOf([
        #         A.OpticalDistortion(),
        #         A.IAAPiecewiseAffine(),
        #     ], p=0.1),
        # A.OneOf([
        #     A.GaussNoise(),
        #     A.MotionBlur(blur_limit=(3, 5)),
        # ], p=0.1),
        # A.ColorJitter(),
        # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        # A.HorizontalFlip(p=0.5),
        # A.Cutout(max_h_size=102, max_w_size=102, num_holes=5, p=0.5),
        # A.CLAHE(clip_limit = 15, p=0.5),
        # albumentations.HorizontalFlip(p=0.5),
        # # albumentations.VerticalFlip(p=0.5),
        # albumentations.RandomBrightness(limit=0.2, p=0.75),
        # albumentations.RandomContrast(limit=0.2, p=0.75),

        # albumentations.OneOf([
        #     albumentations.OpticalDistortion(distort_limit=1.),
        #     albumentations.GridDistortion(num_steps=5, distort_limit=1.),
        # ], p=0.75),

        # albumentations.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=40, val_shift_limit=0, p=0.75),
        # albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=30, border_mode=0, p=0.75),
        # A.Cutout(always_apply=False, p=0.5, num_holes=1, max_h_size=409, max_w_size=409),
        # A.OneOf([ 
        # A.OpticalDistortion(distort_limit=1.0), 
        # A.GridDistortion(num_steps=5, distort_limit=1.),
        # A.ElasticTransform(alpha=3), ], p=0.2),
        # A.OneOf([
        #     # A.GaussNoise(var_limit=[10, 50]),
        #     A.GaussianBlur(),
        #     A.MotionBlur(),
        #     A.MedianBlur(), ], p=0.2),
        # A.OneOf([
        #     A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
        #     A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
        #     A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        # ], p=0.25),
        # A.CoarseDropout(max_holes=8, max_height=image_size//20, max_width=image_size//20,
        #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        # A.Normalize(mean=0, std=1),
        ToTensorV2(),], p=1.0),
    
    "valid": A.Compose([
        # IsotropicResize(max_side =image_size),
        # A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
        # A.Normalize(mean=0, std=1),
        # A.Resize(image_size, image_size),
        ToTensorV2(),
        ], p=1.0)
}

LOGGER.info(f"train transform{data_transforms['train']}")


train transformCompose([
  VerticalFlip(always_apply=False, p=0.5),
  ColorJitter(always_apply=False, p=0.5, brightness=[0.8, 1.2], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=[-0.2, 0.2]),
  ShiftScaleRotate(always_apply=False, p=0.5, shift_limit_x=(-0.0625, 0.0625), shift_limit_y=(-0.0625, 0.0625), scale_limit=(-0.050000000000000044, 0.050000000000000044), rotate_limit=(-10, 10), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box'),
  HorizontalFlip(always_apply=False, p=0.5),
  Cutout(always_apply=False, p=0.5, num_holes=1, max_h_size=409, max_w_size=409),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=False),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


In [7]:
# from albumentations import DualTransform
# image_size = 1024
# def isotropically_resize_image(img, size, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC):
#     h, w = img.shape[:2]
#     if max(w, h) == size:
#         return img
#     if w > h:
#         scale = size / w
#         h = h * scale
#         w = size
#     else:
#         scale = size / h
#         w = w * scale
#         h = size
#     interpolation = interpolation_up if scale > 1 else interpolation_down
#     resized = cv2.resize(img, (int(w), int(h)), interpolation=interpolation)
#     return resized


# class IsotropicResize(DualTransform):
#     def __init__(self, max_side, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC,
#                  always_apply=False, p=1):
#         super(IsotropicResize, self).__init__(always_apply, p)
#         self.max_side = max_side
#         self.interpolation_down = interpolation_down
#         self.interpolation_up = interpolation_up

#     def apply(self, img, interpolation_down=cv2.INTER_AREA, interpolation_up=cv2.INTER_CUBIC, **params):
#         return isotropically_resize_image(img, size=self.max_side, interpolation_down=interpolation_down,
#                                           interpolation_up=interpolation_up)

#     def apply_to_mask(self, img, **params):
#         return self.apply(img, interpolation_down=cv2.INTER_NEAREST, interpolation_up=cv2.INTER_NEAREST, **params)

#     def get_transform_init_args_names(self):
#         return ("max_side", "interpolation_down", "interpolation_up")
    
# data_transforms = {
#     "train": A.Compose([
# #         A.Resize(image_size, image_size),
#         # IsotropicResize(max_side = image_size),
#        A.PadIfNeeded(min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
#         albumentations.HorizontalFlip(p=0.5),
#         # albumentations.VerticalFlip(p=0.5),
#         albumentations.RandomBrightness(limit=0.2, p=0.75),
#         albumentations.RandomContrast(limit=0.2, p=0.75),

#         albumentations.OneOf([
#             albumentations.OpticalDistortion(distort_limit=1.),
#             albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         ], p=0.75),

#         albumentations.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=40, val_shift_limit=0, p=0.75),
#         albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=30, border_mode=0, p=0.75),
#         A.Cutout(always_apply=False, p=0.5, num_holes=1, max_h_size=409, max_w_size=409),
#         # A.RandomBrightnessContrast(),
#         # A.VerticalFlip(p=0.5),   
#         A.ColorJitter(p = 0.7),
#         # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
#         # A.HorizontalFlip(p=0.5),
#         # A.Cutout(max_h_size=int(image_size * 0.1), max_w_size=int(image_size * 0.1), num_holes=5, p=0.5),
#         # albumentations.RandomBrightness(limit=0.2, p=0.75),
#         # albumentations.RandomContrast(limit=0.2, p=0.75),

#         # albumentations.OneOf([
#         #     albumentations.OpticalDistortion(distort_limit=1.),
#         #     albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         # ], p=0.75),

#         # albumentations.HueSaturationValue(hue_shift_limit=40, sat_shift_limit=40, val_shift_limit=0, p=0.75),
#         # albumentations.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.3, rotate_limit=30, border_mode=0, p=0.75),
#         # A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
#         # A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
#         # A.CLAHE(p=0.5),
#         # albumentations.OneOf([
#         # albumentations.OpticalDistortion(distort_limit=1.),
#         # albumentations.GridDistortion(num_steps=5, distort_limit=1.),
#         # ], p=0.75),
#         # A.OneOf([
#         # A.GaussianBlur(),
#         # A.MotionBlur(),
#         # A.MedianBlur(), ], p=0.5),
#         # A.IAASharpen(p = 0.2),
#         # A.JpegCompression(p=0.2),
#         # A.Downscale(scale_min=0.5, scale_max=0.75),
#         # A.OneOf([ A.JpegCompression(), A.Downscale(scale_min=0.1, scale_max=0.15), ], p=0.2), 
#         # A.IAAPiecewiseAffine(),
# #         A.OneOf([ 
# #         A.OpticalDistortion(distort_limit=1.0), 
# #         A.GridDistortion(num_steps=5, distort_limit=1.),
# #         A.ElasticTransform(alpha=3), ], p=0.2),
# #         A.OneOf([
# #             A.GaussNoise(var_limit=[10, 50]),
# #             A.GaussianBlur(),
# #             A.MotionBlur(),
# #             A.MedianBlur(), ], p=0.2),
#         # A.OneOf([
#         #     A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
#         #     A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
#         #     A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
#         # ], p=0.25),
#         # A.CoarseDropout(max_holes=8, max_height=image_size//20, max_width=image_size//20,
#         #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
#         # A.Normalize(mean=0, std=1),
#         ToTensorV2(),], p=1.0),
    
#     "valid": A.Compose([
#         # IsotropicResize(max_side = image_size),
#         A.PadIfNeeded(min_height=image_size, min_width=image_size, border_mode=cv2.BORDER_CONSTANT),
#         # A.Normalize(mean=0, std=1),
# #         A.Resize(image_size, image_size),
#         ToTensorV2(),
#         ], p=1.0)
# }

# LOGGER.info(f"train transform{data_transforms['train']}")


In [8]:
def pad(array, target_shape):
    return np.pad(
        array,
        [(0, target_shape[i] - array.shape[i]) for i in range(len(array.shape))],
        "constant",
    )
    
def load_img(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    image = image[:, :, None]
    # image = cv2.imread(img_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print(image.shape)
    
    # print(image.shape)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image = pad(image, (1024, 800, 3))
        # img = img.reshape((*resize))
    return image
#     image = cv2.resize(image, (320, 320), cv2.INTER_NEAREST)
#     image = image.astype(np.float32)
#     mx = np.max(image)
#     if mx:
#         image/=mx
#     image = image /255.0
    

def load_img1(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image = image[:, :, None]
    return image

def load_img2(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image
class BreastDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_path = f"flip/{row.patient_id}_{row.image_id}.png"
        img = load_img2(img_path)
        label = row['cancer']
        # img = np.transpose(img, (2, 0, 1))
        data = self.transforms(image=img)
        img  = data['image']
        # img = img/255
        return torch.tensor(img).float(), torch.tensor(label).long()
        
    def __len__(self):
        return len(self.df)
    
fold0 = df[df['fold']==0]
train_dataset = BreastDataset(fold0, transforms = data_transforms['train'])
image, label= train_dataset[0]
print(image.shape, label)
# print(image.max())

torch.Size([3, 1344, 840]) tensor(0)


In [9]:

# from pylab import rcParams

# f, axarr = plt.subplots(1,15, figsize = (20, 20))
# imgs = []
# for p in range(15):
#     img, label = train_dataset[p]
#     img = img.transpose(0, 1).transpose(1,2).cpu().numpy()
#     img = img.astype(np.uint8)
#     imgs.append(img)
#     axarr[p].imshow(img)


# Model

In [10]:


class ModelNextVit(nn.Module):
    def __init__(self):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.checkpoint = torch.load('nextvit_small_in1k_384.pth')
        self.backbone = nextvit_small()
        self.backbone.load_state_dict(self.checkpoint['model'])
        self.backbone.proj_head = nn.Linear(1024, 2)

    def forward(self, x):
        x = self.backbone(x)
        return x

class PAM_Module(nn.Module):
    """ Position attention module"""
    #Ref from SAGAN
    def __init__(self, in_dim):
        super(PAM_Module, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X (HxW) X (HxW)
        """
        m_batchsize, C, height, width = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = torch.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


class CAM_Module(nn.Module):
    """ Channel attention module"""
    def __init__(self, in_dim):
        super(CAM_Module, self).__init__()
        self.chanel_in = in_dim
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X H X W)
            returns :
                out : attention value + input feature
                attention: B X C X C
        """
        m_batchsize, C, height, width = x.size()
        proj_query = x.view(m_batchsize, C, -1)
        proj_key = x.view(m_batchsize, C, -1).permute(0, 2, 1)
        energy = torch.bmm(proj_query, proj_key)
        energy_new = torch.max(energy, -1, keepdim=True)[0].expand_as(energy)-energy
        attention = torch.softmax(energy_new, dim=-1)
        proj_value = x.view(m_batchsize, C, -1)

        out = torch.bmm(attention, proj_value)
        out = out.view(m_batchsize, C, height, width)

        out = self.gamma*out + x
        return out


class CBAM(nn.Module):
    def __init__(self, in_channels):
        # def __init__(self):
        super(CBAM, self).__init__()
        inter_channels = in_channels // 4
        self.conv1_c = nn.Sequential(nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(inter_channels),
                                     nn.ReLU())
        
        self.conv1_s = nn.Sequential(nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(inter_channels),
                                     nn.ReLU())

        self.channel_gate = CAM_Module(inter_channels)
        self.spatial_gate = PAM_Module(inter_channels)

        self.conv2_c = nn.Sequential(nn.Conv2d(inter_channels, in_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(in_channels),
                                     nn.ReLU())
        self.conv2_a = nn.Sequential(nn.Conv2d(inter_channels, in_channels, 3, padding=1, bias=False),
                                     nn.BatchNorm2d(in_channels),
                                     nn.ReLU())

    def forward(self, x):
        feat1 = self.conv1_c(x)
        chnl_att = self.channel_gate(feat1)
        chnl_att = self.conv2_c(chnl_att)

        feat2 = self.conv1_s(x)
        spat_att = self.spatial_gate(feat2)
        spat_att = self.conv2_a(spat_att)

        x_out = chnl_att + spat_att

        return x_out
    
class Model(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        self.dropout = nn.Dropout(0.5)
#         self.backbone.classifier = nn.Identity()
        n_features = 1408
        target_size = 2
        self.local_fe = CBAM(n_features)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Sequential(nn.Linear(n_features + n_features, n_features),
                                        nn.BatchNorm1d(n_features),
                                        nn.Dropout(0.1),
                                        nn.ReLU(),
                                        nn.Linear(n_features, target_size))
    def forward(self, x):
        enc_feas = self.backbone.forward_features(x)
        global_feas = F.adaptive_avg_pool2d(enc_feas, 1)
        global_feas = global_feas.reshape(-1, 1408)
        global_feas = self.dropout(global_feas)
#         print(global_feas.shape)
        local_feas = self.local_fe(enc_feas)
        local_feas = torch.sum(local_feas, dim=[2,3])
        local_feas = self.dropout(local_feas)
#         print(local_feas.shape)
        all_feas = torch.cat([global_feas, local_feas], dim=1)
#         print(all_feas.shape)
        x = self.classifier(all_feas)
#         x = self.fc(self.dropout(x))
        return x

class ModelNfNet(nn.Module):
    def __init__(self, model_name='dm_nfnet_f0'):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(model_name, pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.head.fc.in_features,2)
        self.backbone.head.fc = nn.Identity()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x)

class ModelEffNetAttention(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(model_name, pretrained=True,drop_rate = 0.4, drop_path_rate = 0.2)
        self.attention = SpatialAttention()
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        # self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.backbone.forward_features(x)
        att = self.attention(x)
        x = x*att
        x = self.backbone.global_pool(x)
        x = self.fc(self.dropout(x))
        return x

class ModelOld(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        self.dropout = nn.Dropout(0.5)
        self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x
    
class ModelOldOneClass(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.classifier.in_features,1)
        self.dropout = nn.Dropout(0.5)
        self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

class ModelNew(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True,drop_rate = 0.4, drop_path_rate = 0.2)
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        self.dropout = nn.Dropout(0.5)
        self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

class ModelNoDropRate(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True)
        self.fc = nn.Linear(self.backbone.classifier.in_features,2)
        self.dropout = nn.Dropout(0.5)
        self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1.0 / p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        # if p_trainable:
        self.p = nn.Parameter(torch.ones(1) * p)
        # else:
            # self.p = p
        self.eps = eps

    def forward(self, x):
        ret = gem(x, p=self.p, eps=self.eps)
        return ret

    def __repr__(self):
        return (self.__class__.__name__  + f"(p={self.p.data.tolist()[0]:.4f},eps={self.eps})")

class ModelOneChannelAvgGem(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, num_classes = 0, global_pool="",in_chans = 1,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(1408,2)
        self.dropout = nn.Dropout(0.5)
        self.global_pool = GeM()
    def forward(self, x):
        x = self.backbone(x)
        x = self.global_pool(x)
        x = x[:,:,0,0]

        x = self.fc(self.dropout(x))
        return x

class ModelOneChannelAvgPool(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, in_chans = 1,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(1408,2)
        self.dropout = nn.Dropout(0.5)
        self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x

class ModelThreeChannelAvgGem(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, num_classes = 0, global_pool="",in_chans = 3,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(1408,2)
        self.dropout = nn.Dropout(0.5)
        self.global_pool = GeM()
    def forward(self, x):
        x = self.backbone(x)
        x = self.global_pool(x)
        x = x[:,:,0,0]

        x = self.fc(self.dropout(x))
        return x

class ModelThreeChannelAvgPool(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, num_classes = 0, global_pool="",in_chans = 3,drop_rate = 0.3, drop_path_rate = 0.2)
        self.fc = nn.Linear(1408,2)
        self.dropout = nn.Dropout(0.5)
        # self.backbone.classifier = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x[:,:,0,0]

        x = self.fc(self.dropout(x))
        return x

class ModelCVN(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        # ,drop_rate = 0.3, drop_path_rate = 0.2
        self.backbone = timm.create_model('convnext_tiny', pretrained=True,drop_rate = 0.3, drop_path_rate = 0.2, num_classes = 0)
        self.fc = nn.Linear(768,2)
        self.dropout = nn.Dropout(0.5)
        self.backbone.fc = nn.Identity()
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(self.dropout(x))
        return x
# inp = torch.rand(2,3, 1344, 840).to(CFG.device)
# model = ModelCVN(CFG.model_name).to(CFG.device)
# output = model(inp)
# print(output.shape)

In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer, required
import math

class AdamP(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, delta=0.1, wd_ratio=0.1, nesterov=False):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        delta=delta, wd_ratio=wd_ratio, nesterov=nesterov)
        super(AdamP, self).__init__(params, defaults)

    def _channel_view(self, x):
        return x.view(x.size(0), -1)

    def _layer_view(self, x):
        return x.view(1, -1)

    def _cosine_similarity(self, x, y, eps, view_func):
        x = view_func(x)
        y = view_func(y)

        return F.cosine_similarity(x, y, dim=1, eps=eps).abs_()

    def _projection(self, p, grad, perturb, delta, wd_ratio, eps):
        wd = 1
        expand_size = [-1] + [1] * (len(p.shape) - 1)
        for view_func in [self._channel_view, self._layer_view]:

            cosine_sim = self._cosine_similarity(grad, p.data, eps, view_func)

            if cosine_sim.max() < delta / math.sqrt(view_func(p.data).size(1)):
                p_n = p.data / view_func(p.data).norm(dim=1).view(expand_size).add_(eps)
                perturb -= p_n * view_func(p_n * perturb).sum(dim=1).view(expand_size)
                wd = wd_ratio

                return perturb, wd

        return perturb, wd

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad.data
                beta1, beta2 = group['betas']
                nesterov = group['nesterov']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p.data)
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                # Adam
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']

                state['step'] += 1
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']

                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                step_size = group['lr'] / bias_correction1

                if nesterov:
                    perturb = (beta1 * exp_avg + (1 - beta1) * grad) / denom
                else:
                    perturb = exp_avg / denom

                # Projection
                wd_ratio = 1
                if len(p.shape) > 1:
                    perturb, wd_ratio = self._projection(p, grad, perturb, group['delta'], group['wd_ratio'], group['eps'])

                # Weight decay
                if group['weight_decay'] > 0:
                    p.data.mul_(1 - group['lr'] * group['weight_decay'] * wd_ratio)

                # Step
                p.data.add_(perturb, alpha=-step_size)

        return loss

class SGDP(Optimizer):
    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False, eps=1e-8, delta=0.1, wd_ratio=0.1):
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening, weight_decay=weight_decay,
                        nesterov=nesterov, eps=eps, delta=delta, wd_ratio=wd_ratio)
        super(SGDP, self).__init__(params, defaults)

    def _channel_view(self, x):
        return x.view(x.size(0), -1)

    def _layer_view(self, x):
        return x.view(1, -1)

    def _cosine_similarity(self, x, y, eps, view_func):
        x = view_func(x)
        y = view_func(y)

        return F.cosine_similarity(x, y, dim=1, eps=eps).abs_()

    def _projection(self, p, grad, perturb, delta, wd_ratio, eps):
        wd = 1
        expand_size = [-1] + [1] * (len(p.shape) - 1)
        for view_func in [self._channel_view, self._layer_view]:

            cosine_sim = self._cosine_similarity(grad, p.data, eps, view_func)

            if cosine_sim.max() < delta / math.sqrt(view_func(p.data).size(1)):
                p_n = p.data / view_func(p.data).norm(dim=1).view(expand_size).add_(eps)
                perturb -= p_n * view_func(p_n * perturb).sum(dim=1).view(expand_size)
                wd = wd_ratio

                return perturb, wd

        return perturb, wd

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['momentum'] = torch.zeros_like(p.data)

                # SGD
                buf = state['momentum']
                buf.mul_(momentum).add_(grad, alpha=1 - dampening)
                if nesterov:
                    d_p = grad + momentum * buf
                else:
                    d_p = buf

                # Projection
                wd_ratio = 1
                if len(p.shape) > 1:
                    d_p, wd_ratio = self._projection(p, grad, d_p, group['delta'], group['wd_ratio'], group['eps'])

                # Weight decay
                if group['weight_decay'] > 0:
                    p.data.mul_(1 - group['lr'] * group['weight_decay'] * wd_ratio / (1-momentum))

                # Step
                p.data.add_(d_p, alpha=-group['lr'])

        return loss

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [13]:
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau


class GradualWarmupScheduler(_LRScheduler):
    """ Gradually warm-up(increasing) learning rate in optimizer.
    Proposed in 'Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour'.
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        multiplier: target learning rate = base lr * multiplier if multiplier > 1.0. if multiplier = 1.0, lr starts from 0 and ends up with the base_lr.
        total_epoch: target learning rate is reached at total_epoch, gradually
        after_scheduler: after target_epoch, use this scheduler(eg. ReduceLROnPlateau)
    """

    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier < 1.:
            raise ValueError('multiplier should be greater thant or equal to 1.')
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super(GradualWarmupScheduler, self).__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_last_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
        if self.last_epoch <= self.total_epoch:
            warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                param_group['lr'] = lr
        else:
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.total_epoch)

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.total_epoch)
                self._last_lr = self.after_scheduler.get_last_lr()
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

class Lookahead(optim.Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

In [14]:
def log_t(u, t):
    """Compute log_t for `u'."""
    if t==1.0:
        return u.log()
    else:
        return (u.pow(1.0 - t) - 1.0) / (1.0 - t)

def exp_t(u, t):
    """Compute exp_t for `u'."""
    if t==1:
        return u.exp()
    else:
        return (1.0 + (1.0-t)*u).relu().pow(1.0 / (1.0 - t))

def compute_normalization_fixed_point(activations, t, num_iters):

    """Returns the normalization value for each example (t > 1.0).
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (> 1.0 for tail heaviness).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same shape as activation with the last dimension being 1.
    """
    mu, _ = torch.max(activations, -1, keepdim=True)
    normalized_activations_step_0 = activations - mu

    normalized_activations = normalized_activations_step_0

    for _ in range(num_iters):
        logt_partition = torch.sum(
                exp_t(normalized_activations, t), -1, keepdim=True)
        normalized_activations = normalized_activations_step_0 * \
                logt_partition.pow(1.0-t)

    logt_partition = torch.sum(
            exp_t(normalized_activations, t), -1, keepdim=True)
    normalization_constants = - log_t(1.0 / logt_partition, t) + mu

    return normalization_constants

def compute_normalization_binary_search(activations, t, num_iters):

    """Returns the normalization value for each example (t < 1.0).
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (< 1.0 for finite support).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    mu, _ = torch.max(activations, -1, keepdim=True)
    normalized_activations = activations - mu

    effective_dim = \
        torch.sum(
                (normalized_activations > -1.0 / (1.0-t)).to(torch.int32),
            dim=-1, keepdim=True).to(activations.dtype)

    shape_partition = activations.shape[:-1] + (1,)
    lower = torch.zeros(shape_partition, dtype=activations.dtype, device=activations.device)
    upper = -log_t(1.0/effective_dim, t) * torch.ones_like(lower)

    for _ in range(num_iters):
        logt_partition = (upper + lower)/2.0
        sum_probs = torch.sum(
                exp_t(normalized_activations - logt_partition, t),
                dim=-1, keepdim=True)
        update = (sum_probs < 1.0).to(activations.dtype)
        lower = torch.reshape(
                lower * update + (1.0-update) * logt_partition,
                shape_partition)
        upper = torch.reshape(
                upper * (1.0 - update) + update * logt_partition,
                shape_partition)

    logt_partition = (upper + lower)/2.0
    return logt_partition + mu

class ComputeNormalization(torch.autograd.Function):
    """
    Class implementing custom backward pass for compute_normalization. See compute_normalization.
    """
    @staticmethod
    def forward(ctx, activations, t, num_iters):
        if t < 1.0:
            normalization_constants = compute_normalization_binary_search(activations, t, num_iters)
        else:
            normalization_constants = compute_normalization_fixed_point(activations, t, num_iters)

        ctx.save_for_backward(activations, normalization_constants)
        ctx.t=t
        return normalization_constants

    @staticmethod
    def backward(ctx, grad_output):
        activations, normalization_constants = ctx.saved_tensors
        t = ctx.t
        normalized_activations = activations - normalization_constants 
        probabilities = exp_t(normalized_activations, t)
        escorts = probabilities.pow(t)
        escorts = escorts / escorts.sum(dim=-1, keepdim=True)
        grad_input = escorts * grad_output
        
        return grad_input, None, None

def compute_normalization(activations, t, num_iters=5):
    """Returns the normalization value for each example. 
    Backward pass is implemented.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """
    return ComputeNormalization.apply(activations, t, num_iters)

def tempered_sigmoid(activations, t, num_iters = 5):
    """Tempered sigmoid function.
    Args:
      activations: Activations for the positive class for binary classification.
      t: Temperature tensor > 0.0.
      num_iters: Number of iterations to run the method.
    Returns:
      A probabilities tensor.
    """
    internal_activations = torch.stack([activations,
        torch.zeros_like(activations)],
        dim=-1)
    internal_probabilities = tempered_softmax(internal_activations, t, num_iters)
    return internal_probabilities[..., 0]


def tempered_softmax(activations, t, num_iters=5):
    """Tempered softmax function.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      t: Temperature > 1.0.
      num_iters: Number of iterations to run the method.
    Returns:
      A probabilities tensor.
    """
    if t == 1.0:
        return activations.softmax(dim=-1)

    normalization_constants = compute_normalization(activations, t, num_iters)
    return exp_t(activations - normalization_constants, t)

def bi_tempered_binary_logistic_loss(activations,
        labels,
        t1,
        t2,
        label_smoothing = 0.0,
        num_iters=5,
        reduction='mean'):

    """Bi-Tempered binary logistic loss.
    Args:
      activations: A tensor containing activations for class 1.
      labels: A tensor with shape as activations, containing probabilities for class 1
      t1: Temperature 1 (< 1.0 for boundedness).
      t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      label_smoothing: Label smoothing
      num_iters: Number of iterations to run the method.
    Returns:
      A loss tensor.
    """
    internal_activations = torch.stack([activations,
        torch.zeros_like(activations)],
        dim=-1)
    internal_labels = torch.stack([labels.to(activations.dtype),
        1.0 - labels.to(activations.dtype)],
        dim=-1)
    return bi_tempered_logistic_loss(internal_activations, 
            internal_labels,
            t1,
            t2,
            label_smoothing = label_smoothing,
            num_iters = num_iters,
            reduction = reduction)

def bi_tempered_logistic_loss(activations,
        labels,
        t1,
        t2,
        label_smoothing=0.0,
        num_iters=5,
        reduction = 'mean'):

    """Bi-Tempered Logistic Loss.
    Args:
      activations: A multi-dimensional tensor with last dimension `num_classes`.
      labels: A tensor with shape and dtype as activations (onehot), 
        or a long tensor of one dimension less than activations (pytorch standard)
      t1: Temperature 1 (< 1.0 for boundedness).
      t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
      label_smoothing: Label smoothing parameter between [0, 1). Default 0.0.
      num_iters: Number of iterations to run the method. Default 5.
      reduction: ``'none'`` | ``'mean'`` | ``'sum'``. Default ``'mean'``.
        ``'none'``: No reduction is applied, return shape is shape of
        activations without the last dimension.
        ``'mean'``: Loss is averaged over minibatch. Return shape (1,)
        ``'sum'``: Loss is summed over minibatch. Return shape (1,)
    Returns:
      A loss tensor.
    """

    if len(labels.shape)<len(activations.shape): #not one-hot
        labels_onehot = torch.zeros_like(activations)
        labels_onehot.scatter_(1, labels[..., None], 1)
    else:
        labels_onehot = labels

    if label_smoothing > 0:
        num_classes = labels_onehot.shape[-1]
        labels_onehot = ( 1 - label_smoothing * num_classes / (num_classes - 1) ) \
                * labels_onehot + \
                label_smoothing / (num_classes - 1)

    probabilities = tempered_softmax(activations, t2, num_iters)

    loss_values = labels_onehot * log_t(labels_onehot + 1e-10, t1) \
            - labels_onehot * log_t(probabilities, t1) \
            - labels_onehot.pow(2.0 - t1) / (2.0 - t1) \
            + probabilities.pow(2.0 - t1) / (2.0 - t1)
    loss_values = loss_values.sum(dim = -1) #sum over classes

    if reduction == 'none':
        return loss_values
    if reduction == 'sum':
        return loss_values.sum()
    if reduction == 'mean':
        return loss_values.mean()

class BiTemperedLogisticLoss(nn.Module): 
    def __init__(self, t1, t2, smoothing=0.0): 
        super(BiTemperedLogisticLoss, self).__init__() 
        self.t1 = t1
        self.t2 = t2
        self.smoothing = smoothing
    def forward(self, logit_label, truth_label):
        loss_label = bi_tempered_logistic_loss(
            logit_label, truth_label,
            t1=self.t1, t2=self.t2,
            label_smoothing=self.smoothing,
            reduction='none'
        )
        
        loss_label = loss_label.mean()
        return loss_label

In [15]:

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    torch.cuda.empty_cache()
    gc.collect()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    truth = []
    pred = []
    global_step = 0
    scaler = GradScaler()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train')
    for step, (images, labels) in pbar:
        optimizer.zero_grad()
        data_time.update(time.time() - end)
        images = images.to(device)
        
        
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast(enabled=True):
            outputs = model(images)
            loss = criterion(outputs, labels)
            # loss.backward()
            # optimizer.first_step(zero_grad=True)
            # criterion(model(images), labels).backward()
            # optimizer.second_step(zero_grad=True)
            # record loss
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # global_step += 1
        scheduler.step()
            # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # if step % 4 == 0 or step == (len(train_loader)-1):
        #     print('Epoch: [{0}][{1}/{2}] '
        #               'Data {data_time.val:.6f} ({data_time.avg:.6f}) '
        #               'Elapsed {remain:s} '
        #               'Loss: {loss.val:.6f}({loss.avg:.6f}) '
        #               'LR: {lr:.6f}  '
        #               .format(
        #                epoch+1, step, len(train_loader), batch_time=batch_time,
        #                data_time=data_time, loss=losses,
        #                remain=timeSince(start, float(step+1)/len(train_loader)),
        #                lr=scheduler.get_lr()[0],
        #                ))
        torch.cuda.empty_cache()
        gc.collect()
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{losses.avg:0.4f}',
                        lr=f'{current_lr:0.8f}',
                        gpu_mem=f'{mem:0.2f} GB')

    return losses.avg

def valid_fn_no_sigmoid(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append((outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels


def valid_fn(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(torch.sigmoid(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels
def valid_fn_two(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(F.softmax(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels

def valid_fn_two_flip(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        images = torch.flip(images, [3])
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(F.softmax(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels

def valid_fn_flip(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    truth = []
    preds = []
    valid_labels = []
    start = end = time.time()
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        images = torch.flip(images, [3])
        with torch.no_grad():
            outputs = model(images)
        valid_labels.append(labels.cpu().numpy())
        loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(torch.sigmoid(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    valid_labels = np.concatenate(valid_labels)
    return losses.avg, predictions, valid_labels

In [38]:

def pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

def pfbeta_np(labels, preds, beta=1):
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.0
    
def dfs_freeze(module):
    for param in module.parameters():
        param.requires_grad = False
        
def dfs_unfreeze(module):
    for param in module.parameters():
        param.requires_grad = True
    
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

def valid_fn_two(val_dataloader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc='Val')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            with autocast(enabled=True):
                outputs = model(images)
                loss = criterion(outputs, labels)
#         loss = bi_tempered_logistic_loss(outputs, labels, t1=0.8, t2 = 1.4)
        losses.update(loss.item(), batch_size)
#         print(outputs)
        preds.append(F.softmax(outputs).to('cpu').numpy())
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(eval_loss=f'{losses.avg:0.4f}',
                        gpu_mem=f'{mem:0.2f} GB')
    predictions = np.concatenate(preds)
    return losses.avg, predictions
#always seed1, change seed now
# set_seed(1)
seed = 1
LOGGER.info(f"seed: {seed}")
set_seed(seed)
gc.collect()
torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for fold in [2]:
    # LOGGER.info("New dataset")
    # LOGGER.info("site 1")
    LOGGER.info("5 fold")
    LOGGER.info(f"Fold: {fold}")
    LOGGER.info(f"Model name: {CFG.model_name}")
    # model = ModelNfNet().to(device)
    # model = ModelEffNetAttention(model_name=CFG.model_name).to(device)
    # model = ModelOneChannelAvgGem(model_name=CFG.model_name).to(CFG.device)
    
    # model = ModelVIT().to(CFG.device)
    train_df = df1[df1['fold']!=fold].reset_index(drop=True)
    # train_df = train_df[train_df['site_id']==1].reset_index(drop=True)
    valid_df = df[df['fold']==fold].reset_index(drop=True)
    # valid_df = valid_df[valid_df['site_id']==1].reset_index(drop=True)
    # print(len(valid_df))
    LOGGER.info(f"Len train df: {len(train_df)}")
    LOGGER.info(f"Len valid df: {len(valid_df)}")
    train_dataset = BreastDataset(train_df, transforms=data_transforms['train'])

    train_loader = DataLoader(train_dataset, batch_size = CFG.train_bs,
                                  num_workers=1, shuffle=True, drop_last=True)
    
    valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

    valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                  num_workers=1, shuffle=False, drop_last=False)
    # model = Model(model_name=CFG.model_name).to(device)
    LEN_DL_TRAIN = len(train_loader)
    best_f1 = 0
    best_metric = 0
    total_epoch = 15
    warmup = 1
    model = ModelOld(model_name=CFG.model_name).to(device)
    # model = ModelCVN(model_name=CFG.model_name).to(device)
    # checkpoint = torch.load("swa_tf_efficientnetv2_b2_fold_1_model_0.5128_0.3670.pth")
    # model.load_state_dict(checkpoint['state_dict'])
    optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4, weight_decay = 4e-4)
    # optimizer = Lion(model.parameters(), lr = 1e-5, weight_decay = 5e-4)
    # optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup*LEN_DL_TRAIN, num_training_steps =total_epoch*LEN_DL_TRAIN)
    # scheduler = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
    # scheduler.load_state_dict(checkpoint['scheduler'])
    # criterion = nn.BCEWithLogitsLoss().to(CFG.device)
    criterion = nn.CrossEntropyLoss().to(CFG.device)
    LOGGER.info(f"Train bs: {CFG.train_bs}")
    # LOGGER.info(f"Model: {model}")
    LOGGER.info(f"{model.__class__.__name__}")
    LOGGER.info(f"optimizer: {optimizer}")
    LOGGER.info(f"total_epoch :{total_epoch}")
    LOGGER.info(f"Warmup: {warmup}")
    # for epoch in range(1, total_epoch+1):
    #     LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
    #     loss_train = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, CFG.device)
    #     loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    #     # loss_train, train_preds = valid_fn_two(train_loader, model, criterion, CFG.device)
    #     # loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    #     valid_preds = valid_preds[:, 1]
    #     valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    #     valid_preds = np.array(valid_preds).flatten()
        
    #     valid_df['raw_pred'] = valid_preds
    #     # LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    #     LOGGER.info(f"Train loss:{loss_train:.4f}, Valid loss:{loss_valid:.4f}")
    #     grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    #     grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    #     valid_labels_mean = grp_df['cancer'].values
    #     valid_preds_mean = grp_df['raw_pred'].values
    #     # print(valid_labels[:5], valid_preds_mean[:5])
    #     val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    #     LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    #     best_metric_mean_at_epoch = 0
    #     best_metric1 = 0
    #     best_threshold_mean = 0
    #     best_auc = 0
    #     best_cf = None
    #     for i in np.arange(0.001, 0.599, 0.001):
    #         valid_argmax = (valid_preds_mean>i).astype(np.int32)
    # #             print(valid_argmax)
    #         # val_metric = pfbeta(valid_labels_mean, valid_argmax)
    #         val_metric1 = pfbeta_np(valid_labels_mean, valid_argmax)
    #         val_acc = accuracy_score(valid_labels_mean, valid_argmax)
    #         val_f1 = f1_score(valid_labels_mean, valid_argmax)
    #         val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
    #         cf = confusion_matrix(valid_labels_mean, valid_argmax)
    #         if val_metric1> best_metric1:
    #             best_metric1 = val_metric1
    #             # best_metric_mean_at_epoch = val_metric
    #             best_threshold_mean = i
    #             best_auc = val_auc
    #             best_cf = cf
    #     LOGGER.info(f"Best metric at epoch {epoch}: {best_metric1:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    #     LOGGER.info(f"Cf: {best_cf}")
    #     if best_metric1> best_metric:

    #         LOGGER.info(f"Model improve: {best_metric:.4f} -> {best_metric1:.4f}")
    #         best_metric = best_metric1
    #     state = {'epoch': epoch, 'state_dict': model.state_dict()}
    #     # state = {'epoch': epoch, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict(), 'scheduler':scheduler.state_dict()}
    #     path = f'fold{fold}/{CFG.model_name}_fold_{fold}_model_epoch_{epoch}_{best_metric1:.4f}_{best_threshold_mean:.3f}.pth'
    #     torch.save(state, path)

        

seed: 1
5 fold
Fold: 2
Model name: tf_efficientnetv2_b2
Len train df: 46421
Len valid df: 11012


> SEEDING DONE


Train bs: 16
ModelOld
optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.0001
    lr: 0.0
    maximize: False
    weight_decay: 0.0004
)
total_epoch :15
Warmup: 1


In [39]:
# for fold in [0]:
#     valid_df = df[df['fold']==fold].reset_index(drop=True)   
#     valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

#     valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
#                                   num_workers=1, shuffle=False, drop_last=False)
#     # model = Model(model_name=CFG.model_name).to(device)
#     LEN_DL_TRAIN = len(train_loader)
#     best_f1 = 0
#     best_metric = 0
#     total_epoch = 15
#     warmup = 1
#     model = ModelOld(model_name=CFG.model_name).to(device)
#     checkpoint = torch.load("foldsensemble/swa_tf_efficientnetv2_b2_fold_0_model_0.5269_0.3540.pth")
#     model.load_state_dict(checkpoint['state_dict'])
#     for epoch in range(1, 2):
#         LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
#         loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         # loss_train, train_preds = valid_fn_two(train_loader, model, criterion, CFG.device)
#         # loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         valid_preds = valid_preds[:, 1]
#         valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
#         valid_preds = np.array(valid_preds).flatten()
        
#         valid_df['raw_pred'] = valid_preds

In [40]:
# for fold in [1]:
#     valid_df1 = df[df['fold']==fold].reset_index(drop=True)   
#     valid_dataset = BreastDataset(valid_df1, transforms=data_transforms['valid'])

#     valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
#                                   num_workers=1, shuffle=False, drop_last=False)
#     # model = Model(model_name=CFG.model_name).to(device)
#     LEN_DL_TRAIN = len(train_loader)
#     best_f1 = 0
#     best_metric = 0
#     total_epoch = 15
#     warmup = 1
#     model = ModelOld(model_name=CFG.model_name).to(device)
#     checkpoint = torch.load("swa_tf_efficientnetv2_b2_fold_1_model_0.5078_0.3050.pth")
#     model.load_state_dict(checkpoint['state_dict'])
#     for epoch in range(1, 2):
#         LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
#         loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         # loss_train, train_preds = valid_fn_two(train_loader, model, criterion, CFG.device)
#         # loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         valid_preds = valid_preds[:, 1]
#         valid_df1['prediction_id'] = valid_df1['patient_id'].astype(str) + '_' + valid_df1['laterality'].astype(str)
#         valid_preds = np.array(valid_preds).flatten()
        
#         valid_df1['raw_pred'] = valid_preds

In [41]:
# for fold in [2]:
#     valid_df2 = df[df['fold']==fold].reset_index(drop=True)   
#     valid_dataset = BreastDataset(valid_df2, transforms=data_transforms['valid'])

#     valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
#                                   num_workers=1, shuffle=False, drop_last=False)
#     # model = Model(model_name=CFG.model_name).to(device)
#     LEN_DL_TRAIN = len(train_loader)
#     best_f1 = 0
#     best_metric = 0
#     total_epoch = 15
#     warmup = 1
#     model = ModelOld(model_name=CFG.model_name).to(device)
#     checkpoint = torch.load("foldsensemble/swa_tf_efficientnetv2_b2_fold_2_model_0.5281_0.454.pth")
#     model.load_state_dict(checkpoint['state_dict'])
#     for epoch in range(1, 2):
#         LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
#         loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         # loss_train, train_preds = valid_fn_two(train_loader, model, criterion, CFG.device)
#         # loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         valid_preds = valid_preds[:, 1]
#         valid_df2['prediction_id'] = valid_df2['patient_id'].astype(str) + '_' + valid_df2['laterality'].astype(str)
#         valid_preds = np.array(valid_preds).flatten()
        
#         valid_df2['raw_pred'] = valid_preds

In [42]:
# for fold in [3]:
#     valid_df3 = df[df['fold']==fold].reset_index(drop=True)   
#     valid_dataset = BreastDataset(valid_df3, transforms=data_transforms['valid'])

#     valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
#                                   num_workers=1, shuffle=False, drop_last=False)
#     # model = Model(model_name=CFG.model_name).to(device)
#     LEN_DL_TRAIN = len(train_loader)
#     best_f1 = 0
#     best_metric = 0
#     total_epoch = 15
#     warmup = 1
#     model = ModelOld(model_name=CFG.model_name).to(device)
#     checkpoint = torch.load("swa_tf_efficientnetv2_b2_fold_3_model_0.5325_0.428.pth")
#     model.load_state_dict(checkpoint['state_dict'])
#     for epoch in range(1, 2):
#         LOGGER.info(f"Epoch: {epoch}/{total_epoch}")
#         loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         # loss_train, train_preds = valid_fn_two(train_loader, model, criterion, CFG.device)
#         # loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#         valid_preds = valid_preds[:, 1]
#         valid_df3['prediction_id'] = valid_df3['patient_id'].astype(str) + '_' + valid_df3['laterality'].astype(str)
#         valid_preds = np.array(valid_preds).flatten()
        
#         valid_df3['raw_pred'] = valid_preds

In [43]:
# final_valid_df = pd.concat([valid_df, valid_df1, valid_df2, valid_df3], axis = 0).reset_index(drop=True)
# print(final_valid_df.head())

In [44]:
# print(len(final_valid_df))

In [45]:
# from sklearn import metrics
# def np_binary_cross_entropy_loss(probability, truth):
#     probability = probability.astype(np.float64)
#     probability = np.nan_to_num(probability, nan=1, posinf=1, neginf=0)

#     p = np.clip(probability, 1e-5, 1 - 1e-5)
#     y = truth

#     loss = -y * np.log(p) - (1 - y) * np.log(1 - p)
#     loss = loss.mean()
#     return loss

# def get_f1score(probability, truth, threshold = np.linspace(0.2, 0.5, 10000)):
#     f1score = []
#     precision=[]
#     recall=[]
#     for t in threshold:
#         predict = (probability > t).astype(np.float32)

#         tp = ((predict >= 0.5) & (truth >= 0.5)).sum()
#         fp = ((predict >= 0.5) & (truth < 0.5)).sum()
#         fn = ((predict < 0.5) & (truth >= 0.5)).sum()

#         r = tp / (tp + fn + 1e-3)
#         p = tp / (tp + fp + 1e-3)
#         f1 = 2 * r * p / (r + p + 1e-3)
#         f1score.append(f1)
#         precision.append(p)
#         recall.append(r)
#     f1score = np.array(f1score)
#     precision = np.array(precision)
#     recall = np.array(recall)
#     return f1score, precision, recall, threshold


# def compute_metric(cancer_p, cancer_t):

#     fpr, tpr, thresholds = metrics.roc_curve(cancer_t, cancer_p)
#     auc = metrics.auc(fpr, tpr)

#     f1score, precision, recall, threshold = get_f1score(cancer_p, cancer_t)
#     i = f1score.argmax()
#     f1score, precision, recall, threshold = f1score[i], precision[i], recall[i], threshold[i]

#     specificity = ((cancer_p < threshold ) & ((cancer_t <= 0.5))).sum() / (cancer_t <= 0.5).sum()
#     sensitivity = ((cancer_p >= threshold) & ((cancer_t >= 0.5))).sum() / (cancer_t >= 0.5).sum()

#     return {
#         'auc': auc,
#         'threshold': threshold,
#         'f1score': f1score,
#         'precision': precision,
#         'recall': recall,
#         'sensitivity': sensitivity,
#         'specificity': specificity,
#     }

# def compute_pfbeta(labels, predictions, beta=1):
#     y_true_count = 0
#     ctp = 0
#     cfp = 0

#     for idx in range(len(labels)):
#         prediction = min(max(predictions[idx], 0), 1)
#         if (labels[idx]):
#             y_true_count += 1
#             ctp += prediction
#             #cfp += 1 - prediction
#         else:
#             cfp += prediction

#     beta_squared = beta * beta
#     c_precision = ctp / (ctp + cfp+1e-8)
#     c_recall = ctp / y_true_count
#     if (c_precision > 0 and c_recall > 0):
#         result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
#         return result
#     else:
#         return 0

# def print_all_metric(valid_df):

# 	print(f'{"    ": <16}    \tauc      @th     f1      | 	prec    recall  | 	sens    spec ')
# 	#log.write(f'{"    ": <16}    \t0.77902	0.44898	0.28654 | 	0.32461	0.25726 | 	0.25726	0.98794\n')
# 	for site_id in [0,1,2]:

# 		#log.write(f'*** site_id [{site_id}] ***\n')
# 		#log.write(f'\n')

# 		if site_id>0:
# 			site_df = valid_df[valid_df.site_id == site_id].reset_index(drop=True)
# 		else:
# 			site_df = valid_df
# 		# ---

# 		gb = site_df
# 		m = compute_metric(gb.raw_pred, gb.cancer)
# 		text = f'{"single image": <16} [{site_id}]'
# 		text += f'\t{m["auc"]:0.5f}'
# 		text += f'\t{m["threshold"]:0.5f}'
# 		text += f'\t{m["f1score"]:0.5f} | '
# 		text += f'\t{m["precision"]:0.5f}'
# 		text += f'\t{m["recall"]:0.5f} | '
# 		text += f'\t{m["sensitivity"]:0.5f}'
# 		text += f'\t{m["specificity"]:0.5f}'
# 		#text += '\n'
# 		print(text)


# 		# ---

# 		gb = site_df[['patient_id', 'laterality', 'cancer', 'raw_pred']].groupby(['patient_id', 'laterality']).mean()
# 		m = compute_metric(gb.raw_pred, gb.cancer)
# 		text = f'{"grouby mean()": <16} [{site_id}]'
# 		text += f'\t{m["auc"]:0.5f}'
# 		text += f'\t{m["threshold"]:0.5f}'
# 		text += f'\t{m["f1score"]:0.5f} | '
# 		text += f'\t{m["precision"]:0.5f}'
# 		text += f'\t{m["recall"]:0.5f} | '
# 		text += f'\t{m["sensitivity"]:0.5f}'
# 		text += f'\t{m["specificity"]:0.5f}'
# 		#text += '\n'
# 		print(text)

# 		# ---
# 		gb = site_df[['patient_id', 'laterality', 'cancer', 'raw_pred']].groupby(['patient_id', 'laterality']).max()
# 		m = compute_metric(gb.raw_pred, gb.cancer)
# 		text = f'{"grouby max()": <16} [{site_id}]'
# 		text += f'\t{m["auc"]:0.5f}'
# 		text += f'\t{m["threshold"]:0.5f}'
# 		text += f'\t{m["f1score"]:0.5f} | '
# 		text += f'\t{m["precision"]:0.5f}'
# 		text += f'\t{m["recall"]:0.5f} | '
# 		text += f'\t{m["sensitivity"]:0.5f}'
# 		text += f'\t{m["specificity"]:0.5f}'
# 		#text += '\n'
# 		print(text)
# 		print(f'--------------\n')


# # valid_df.loc[:, 'cancer_t'] = valid_preds
# print_all_metric(final_valid_df)

In [46]:

# grp_df = final_valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
# grp_df['cancer'] = grp_df['cancer'].astype(np.int)
# valid_labels_mean = grp_df['cancer'].values
# valid_preds_mean = grp_df['raw_pred'].values
# # print(valid_labels[:5], valid_preds_mean[:5])
# val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
# LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
# best_metric_mean_at_epoch = 0
# best_metric1 = 0
# best_threshold_mean = 0
# best_auc = 0
# best_cf = None
# for i in np.arange(0.001, 0.599, 0.001):
#     valid_argmax = (valid_preds_mean>i).astype(np.int32)
# #             print(valid_argmax)
#     # val_metric = pfbeta(valid_labels_mean, valid_argmax)
#     val_metric1 = pfbeta_np(valid_labels_mean, valid_argmax)
#     val_acc = accuracy_score(valid_labels_mean, valid_argmax)
#     val_f1 = f1_score(valid_labels_mean, valid_argmax)
#     val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
#     cf = confusion_matrix(valid_labels_mean, valid_argmax)
#     if val_metric1> best_metric1:
#         best_metric1 = val_metric1
#         # best_metric_mean_at_epoch = val_metric
#         best_threshold_mean = i
#         best_auc = val_auc
#         best_cf = cf
# LOGGER.info(f"Best metric at epoch {epoch}: {best_metric1:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")

In [36]:
import optuna
from optuna.samplers import TPESampler

In [37]:

# set_seed(1)
# out_file = 'swa_triple_ensemble_model_fold0_5.pth' 
# iteration = [
#     # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth',
#     # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.4757_0.230.pth',
#     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth',
#     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth',
    
#     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth',
#     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth',
#     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth',
    
#     # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_8_0.4403_0.415.pth',
# #     'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_11_0.4387_0.436.pth'
# ]

# criterion = nn.CrossEntropyLoss().to(CFG.device)
# best_metric = 0
# torch.cuda.empty_cache()
# def objective(trial):
# #     a1 = 0.036839841333967636 
# #     a2 = 0.6490629183820655
# #     a3 = 0.3140972402839668
# #     a2 = 0.47142151346976024 
# #     a3 = 0.3596277792186039
# #     a1 = trial.suggest_uniform('a1', 0.01, 0.99)
# #     a2 = 1-a1
#     a1 = trial.suggest_uniform('a1', 0.001, 0.99)
#     # a2 = 1-a1
#     a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
#     a3 = trial.suggest_uniform('a3', 0.00009, 1-a1-a2-0.001)
#     # a4 = 1-a1-a2-a3
#     a4 = trial.suggest_loguniform('a4', 0.000009, 1-a1-a2-a3-0.001)
#     a5 = 1-a1-a2-a3-a4
#     # a5 = trial.suggest_loguniform('a5', 0.0000009, 1-a1-a2-a3-a4-0.001)
#     a6 = 1-a1-a2-a3-a4-a5
# #     a5 = trial.suggest_loguniform('a5', 0.000009, 1-a1-a2-a3-a4-0.001)
# #     a6 = trial.suggest_loguniform('a6', 0.0000009, 1-a1-a2-a3-a4-a5-0.001)
# #     a7 = 1-a1-a2-a3-a4-a5-a6
#     state_dict = None
#     for i in iteration:
#         f = i
#         f = torch.load(f, map_location=lambda storage, loc: storage)
#         if state_dict is None:
#             print("none: ", i)
#             state_dict = f['state_dict']
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = f['state_dict'][k]*a1
#         elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
#         elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
                
#         elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
#         elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
#         # elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth':
#         #     print("noob", i)
#         #     key = list(f['state_dict'].keys())
#         #     for k in key:
#         #         state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
#     print(a1, a2, a3, a4, a5)
#     # for k in key:
#     #     state_dict[k] = state_dict[k] / len(iteration)
#     print('')

#     # print(out_file)
#     torch.save({'state_dict': state_dict}, out_file)

#     model = ModelOld(model_name=CFG.model_name).to(CFG.device)
#     checkpoint = torch.load("swa_triple_ensemble_model_fold0_5.pth")
#     model.load_state_dict(checkpoint['state_dict'])
# #     model = nn.DataParallel(model)

#     loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#     valid_preds = valid_preds[:, 1]
#     valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
#     valid_preds = np.array(valid_preds).flatten()
    
#     valid_df['raw_pred'] = valid_preds
#     LOGGER.info(f"Valid loss:{loss_valid:.4f}")
#     grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
#     grp_df['cancer'] = grp_df['cancer'].astype(np.int)
#     valid_labels_mean = grp_df['cancer'].values
#     valid_preds_mean = grp_df['raw_pred'].values
#     # print(valid_labels[:5], valid_preds_mean[:5])
#     val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
#     LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
#     best_metric_mean_at_epoch = 0
#     best_metric = 0
    
#     best_threshold_mean = 0
#     best_auc = 0
#     best_cf = None
#     for i in np.arange(0.001, 0.599, 0.001):
#         valid_argmax = (valid_preds_mean>i).astype(np.int32)
#         val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
#         val_acc = accuracy_score(valid_labels_mean, valid_argmax)
#         val_f1 = f1_score(valid_labels_mean, valid_argmax)
#         val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
#         cf = confusion_matrix(valid_labels_mean, valid_argmax)
#         if val_metric> best_metric:
#             best_metric = val_metric
#             # best_metric_mean_at_epoch = val_metric
#             best_threshold_mean = i
#             best_auc = val_auc
#             best_cf = cf
#     if best_metric>0.5269:
#         state = {'state_dict': model.state_dict()}
#         path = f'swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.4f}.pth'
#         torch.save(state, path)
    
#     LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
#     LOGGER.info(f"Cf: {best_cf}")
#     return best_metric

# study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=777))
# study.optimize(func=objective, n_trials=1000)
# study.best_params
# # # 0.5563409550491111 0.4436590449508889 fold 0
# # # 0.12634002523631388 0.8351954705276587 0.03846450423602743 0.5393 
# # # 0.583301614081906 0.3673525472043472 0.04934583871374687 fold 2 0.50
# # # 0.1689507073116359 0.47142151346976024 0.3596277792186039 fold 2 0.5055 0.5055 0.3670  0.7261

[I 2023-02-24 15:00:07,570] A new study created in memory with name: no-name-53710a50-c91b-403f-8002-fe3c3577b842


> SEEDING DONE
none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth
0.15198443381740748 0.25672863401172846 0.03670370164741394 0.0014337924369785331 0.5531494380864715



Val: 100%|██████████| 172/172 [03:00<00:00,  1.05s/it, eval_loss=0.0860, gpu_mem=7.81 GB]
Valid loss:0.0860
Val metric mean prob: 0.2583
Best metric at: 0.5057 0.3750  0.7168
Cf: [[4636   30]
 [  56   44]]
[I 2023-02-24 15:04:12,869] Trial 0 finished with value: 0.5057471264367815 and parameters: {'a1': 0.15198443381740748, 'a2': 0.25672863401172846, 'a3': 0.03670370164741394, 'a4': 0.0014337924369785331}. Best is trial 0 with value: 0.5057471264367815.


none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth
0.8270655972088143 0.15944838645109546 0.009101767328684767 0.0008576087380599103 0.0035266402733455183



Val: 100%|██████████| 172/172 [03:05<00:00,  1.08s/it, eval_loss=0.1085, gpu_mem=7.81 GB]
Valid loss:0.1085
Val metric mean prob: 0.1980
Best metric at: 0.5087 0.4910  0.7169
Cf: [[4637   29]
 [  56   44]]
[I 2023-02-24 15:07:26,810] Trial 1 finished with value: 0.508670520231214 and parameters: {'a1': 0.8270655972088143, 'a2': 0.15944838645109546, 'a3': 0.009101767328684767, 'a4': 0.0008576087380599103}. Best is trial 1 with value: 0.508670520231214.


...

none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth
0.756601740646172 0.22323819805627493 0.007053719243254243 0.00018265190986090887 0.012923690144437892



Val: 100%|██████████| 172/172 [03:02<00:00,  1.06s/it, eval_loss=0.1074, gpu_mem=7.81 GB]
Valid loss:0.1074
Val metric mean prob: 0.1962
Best metric at: 0.5172 0.4850  0.7219
Cf: [[4637   29]
 [  55   45]]
[I 2023-02-24 23:28:11,849] Trial 156 finished with value: 0.5172413793103449 and parameters: {'a1': 0.756601740646172, 'a2': 0.22323819805627493, 'a3': 0.007053719243254243, 'a4': 0.00018265190986090887}. Best is trial 133 with value: 0.5287356321839081.


none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_3_0.4945_0.488.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth
0.7174932742526814 0.2377459576216479 0.01804071900649787 0.0002212485903798395 0.026498800528792995



Val: 100%|██████████| 172/172 [03:05<00:00,  1.08s/it, eval_loss=0.1068, gpu_mem=7.81 GB]
Valid loss:0.1068
Val metric mean prob: 0.1957
Best metric at: 0.5287 0.4820  0.7270
Cf: [[4638   28]
 [  54   46]]
[I 2023-02-24 23:31:29,343] Trial 157 finished with value: 0.5287356321839081 and parameters: {'a1': 0.7174932742526814, 'a2': 0.2377459576216479, 'a3': 0.01804071900649787, 'a4': 0.0002212485903798395}. Best is trial 133 with value: 0.5287356321839081.
[W 2023-02-24 23:31:29,354] Trial 158 failed with parameters: {'a1': 0.8111272587592091, 'a2': 0.1854941131543417, 'a3': 0.032858629785276094} because of the following error: ValueError('The `low` value must be smaller than or equal to the `high` value (low=9e-06, high=-0.030480001698826892).').
Traceback (most recent call last):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_30113/2110198947.py", lin

ValueError: The `low` value must be smaller than or equal to the `high` value (low=9e-06, high=-0.030480001698826892).

# Fold 1/10

In [47]:
def pfbeta_np(labels, preds, beta=1):
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.0
# fold=3
# valid_df = df[df['fold']==fold].reset_index(drop=True)
# valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                num_workers=1, shuffle=False, drop_last=False)
set_seed(1)
out_file = 'swa_model_fold2_5.pth' 
iteration = [
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4585_0.236.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_7_0.4557_0.241.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_10_0.4550_0.245.pth',
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
#     a2 = 0.12003546043452194 
#     a3 = 0.8649578775769542
#     a1 = 0.020317850755860567 
#     a2 = 0.1293785181217534 
#     a3 = 0.850303631122386
    # a1 = 0.2
    # a2 = 0.2    
    # a3 = 0.2
    # a4 = 0.2
    # a5 = 0.2
    # a1 = 1
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = trial.suggest_uniform('a3', 0.0009, 1-a1-a2-0.001)
    a4 = 1-a1-a2-a3
    # a4 = trial.suggest_loguniform('a4', 0.0009, 1-a1-a2-a3-0.001)
    # a5 = 1-a1-a2-a3-a4
    # a5 = trial.suggest_loguniform('a5', 0.00009, 1-a1-a2-a3-a4-0.001)
    # a6 = 1-a1-a2-a3-a4-a5
    # a4 = 1-a1-a2-a3
    # a1 = 0.4700450486328235 
    # a2 = 0.23862687145742947 
    # a3 = 0.2913280799097471
    state_dict = None
    for i in iteration:
        f = i
        # print(f)
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4585_0.236.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
        # elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_7_0.4557_0.241.pth':
        #     print("noobie", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
        # elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_10_0.4550_0.245.pth':
        #     print("noobie", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
    print(a1, a2, a3, a4)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = ModelOld(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold2_5.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    if best_metric>0.52:
        state = {'state_dict': model.state_dict()}
        path = f'swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.3f}.pth'
        torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    torch.cuda.empty_cache()
    return best_metric

study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=777))
study.optimize(func=objective, n_trials=200)
study.best_params

[I 2023-02-24 23:41:25,945] A new study created in memory with name: no-name-e21dbbda-5be0-464b-9149-8bf3d3e52735


> SEEDING DONE
none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth


# Fold 0/10

In [ ]:

set_seed(1)
out_file = 'swa_model_fold0_10.pth' 
iteration = [
    'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.5825_0.137.pth',
    'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth',
    'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth', 
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
    # a1 = 0.036839841333967636 
    # a2 = 0.6490629183820655
    # a3 = 0.3140972402839668
    # a2 = 0.47142151346976024 
    # a3 = 0.3596277792186039
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    # a2 = 1-a1
    # a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = 1-a1-a2
    # a3 = trial.suggest_uniform('a3', 0.0009, 1-a1-a2-0.001)
    # a4 = 1-a1-a2-a3
    # a4 = trial.suggest_uniform('a4', 0.0009, 1-a1-a2-a3-0.001)
    # a5 = 1-a1-a2-a3-a4
    # a5_lower = max(0.0009, 1-a1-a2-a3-a4-0.999)
    # a5 = trial.suggest_loguniform('a5', a5_lower, 1-a1-a2-a3-a4-0.001)
    # a6  =1-a1-a2-a3-a4-a5
    # a6 = trial.suggest_loguniform('a6', 0.00006, 1-a1-a2-a3-a4-a5)
    # a7 = 1-a1-a2-a3-a4-a5-a6
    # a1 = 0.24757534134332052 
    # a2 = 0.10196581218020337 
    # a3 = 0.08626526314257006 
    # a4 = 0.09079196656355407 
    # a5 = 0.473401616770352
    state_dict = None
    for i in iteration:
        f = i
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
        # elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_2_0.5510_0.151.pth':
        #     print("noob", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
                
        # elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth':
        #     print("noob", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
        # elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth':
        #     print("noob", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
        # elif i=='fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth':
        #     print("noob", i)
        #     key = list(f['state_dict'].keys())
        #     for k in key:
        #         state_dict[k] = state_dict[k] + a7*f['state_dict'][k]
    print(a1, a2, a3)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = ModelOld(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold0_10.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    if best_metric > 0.61:
        state = {'state_dict': model.state_dict()}
        path = f'swa_diffaug_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.4f}.pth'
        torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    return best_metric
# seed 777 fold 3
study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=777))
study.optimize(func=objective, n_trials=200)
study.best_params
# # 0.5563409550491111 0.4436590449508889 fold 0
# # 0.12634002523631388 0.8351954705276587 0.03846450423602743 0.5393 
# # 0.583301614081906 0.3673525472043472 0.04934583871374687 fold 2 0.50
# # 0.1689507073116359 0.47142151346976024 0.3596277792186039 fold 2 0.5055 0.5055 0.3670  0.7261

[I 2023-02-20 02:57:33,097] A new study created in memory with name: no-name-3aa99082-12ec-4dbc-a712-bb22940d30a2


> SEEDING DONE
none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.5825_0.137.pth
hehe fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth
0.15198443381740748 0.25672863401172846 0.591286932170864



Val: 100%|██████████| 86/86 [01:31<00:00,  1.06s/it, eval_loss=0.0752, gpu_mem=9.00 GB]
Valid loss:0.0752
Val metric mean prob: 0.3495
Best metric at: 0.5783 0.4180  0.7428
Cf: [[2324   10]
 [  25   24]]
[I 2023-02-20 02:59:12,325] Trial 0 finished with value: 0.5783132530120482 and parameters: {'a1': 0.15198443381740748, 'a2': 0.25672863401172846}. Best is trial 0 with value: 0.5783132530120482.


none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.5825_0.137.pth
hehe fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth
0.06235401415270182 0.43121246871095986 0.5064335171363383



Val: 100%|██████████| 86/86 [01:24<00:00,  1.02it/s, eval_loss=0.0807, gpu_mem=9.00 GB]
Valid loss:0.0807
Val metric mean prob: 0.3465
Best metric at: 0.5500 0.4880  0.7226
Cf: [[2325    9]
 [  27   22]]
[I 2023-02-20 03:00:44,629] Trial 1 finished with value: 0.5499999999999999 and parameters: {'a1': 0.06235401415270182, 'a2': 0.43121246871095986}. Best is trial 0 with value: 0.5783132530120482.


...

none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.5825_0.137.pth
hehe fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth
0.9643696763259075 0.0042684483938768 0.03136187528021567



Val: 100%|██████████| 86/86 [01:24<00:00,  1.01it/s, eval_loss=0.0680, gpu_mem=9.00 GB]
Valid loss:0.0680
Val metric mean prob: 0.2808
Best metric at: 0.5895 0.1590  0.7819
Cf: [[2316   18]
 [  21   28]]
[I 2023-02-20 03:25:43,325] Trial 17 finished with value: 0.5894736842105263 and parameters: {'a1': 0.9643696763259075, 'a2': 0.0042684483938768}. Best is trial 3 with value: 0.6265060240963856.


none:  fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.5825_0.137.pth
hehe fold0/tf_efficientnetv2_b2_fold_0_model_epoch_13_0.5750_0.437.pth
noob fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5679_0.380.pth
0.8456044398642144 0.05556661187953133 0.09882894825625427



[W 2023-02-20 03:25:46,784] Trial 18 failed with parameters: {'a1': 0.8456044398642144, 'a2': 0.05556661187953133} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_12286/3795447351.py", line 108, in objective
    checkpoint = torch.load("swa_model_fold0_10.pth")
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/torch/serialization.py", line 772, in load
    if _is_zipfile(opened_file):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/torch/serialization.py", line 78, in _is_zipfile
    byte = f.read(1)
KeyboardInterrupt
[W 2023-02-20 03:25:46,800] Trial 18 failed with value None.


KeyboardInterrupt: 

# Fold 2

In [ ]:
fold=2
valid_df = df[df['fold']==fold].reset_index(drop=True)
valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                num_workers=1, shuffle=False, drop_last=False)
set_seed(1)
out_file = 'swa_model_fold2_5.pth' 
iteration = [
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth',
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
#     a1 = 0.4360187712961733
#     a2 = 0.31005592868022136
#     a3 = 0.25392530002360536
#     a1 = 0.015006661988523864 
#     a2 = 0.12003546043452194 
#     a3 = 0.8649578775769542
#     a1 = 0.020317850755860567 
#     a2 = 0.1293785181217534 
#     a3 = 0.850303631122386
#     a1 = 0.12634002523631388
#     a2 = 0.8351954705276587
#     a3 = 0.03846450423602743
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = 1-a1-a2
    # a3 = trial.suggest_uniform('a3', 0.003, 1-a1-a2)
    # a1 = 0.4700450486328235 
    # a2 = 0.23862687145742947 
    # a3 = 0.2913280799097471
    state_dict = None
    for i in iteration:
        f = i
        # print(f)
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
#         elif i=='fold1/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth':
#             print("noobie", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
    print(a1, a2, a3)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = ModelOld(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold2_5.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    if best_metric>0.47:
        state = {'state_dict': model.state_dict()}
        path = f'site1_swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.3f}.pth'
        torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    return best_metric

study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=777))
study.optimize(func=objective, n_trials=200)
study.best_params

[I 2023-02-18 00:12:15,175] A new study created in memory with name: no-name-1cf6bd6d-d453-4b38-99c1-19a712c38d45


> SEEDING DONE
none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
noob fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth
0.15198443381740748 0.25672863401172846 0.591286932170864



Val: 100%|██████████| 96/96 [01:38<00:00,  1.03s/it, eval_loss=0.1082, gpu_mem=8.82 GB]
Valid loss:0.1082
Val metric mean prob: 0.2358
Best metric at: 0.4390 0.4660  0.6737
Cf: [[2334   13]
 [  33   18]]
[I 2023-02-18 00:14:04,971] Trial 0 finished with value: 0.4390243902439025 and parameters: {'a1': 0.15198443381740748, 'a2': 0.25672863401172846}. Best is trial 0 with value: 0.4390243902439025.


none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
noob fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth
0.06235401415270182 0.43121246871095986 0.5064335171363383



Val: 100%|██████████| 96/96 [01:36<00:00,  1.01s/it, eval_loss=0.1057, gpu_mem=8.82 GB]
Valid loss:0.1057
Val metric mean prob: 0.2324
Best metric at: 0.4337 0.4290  0.6735
Cf: [[2333   14]
 [  33   18]]
[I 2023-02-18 00:15:49,759] Trial 1 finished with value: 0.43373493975903615 and parameters: {'a1': 0.06235401415270182, 'a2': 0.43121246871095986}. Best is trial 0 with value: 0.4390243902439025.


...

none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
noob fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth
0.5426633616420505 0.21679443361663897 0.2405422047413105



Val: 100%|██████████| 96/96 [01:36<00:00,  1.00s/it, eval_loss=0.1020, gpu_mem=8.82 GB]
Valid loss:0.1020
Val metric mean prob: 0.2328
Best metric at: 0.4524 0.4590  0.6833
Cf: [[2333   14]
 [  32   19]]
[I 2023-02-18 05:02:27,149] Trial 159 finished with value: 0.45238095238095244 and parameters: {'a1': 0.5426633616420505, 'a2': 0.21679443361663897}. Best is trial 61 with value: 0.4819277108433735.


none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
noob fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth
0.4324526077797173 0.189150277238796 0.37839711498148676



Val:  53%|█████▎    | 51/96 [00:51<00:45,  1.01s/it, eval_loss=0.1176, gpu_mem=8.82 GB]
[W 2023-02-18 05:03:25,677] Trial 160 failed with parameters: {'a1': 0.4324526077797173, 'a2': 0.189150277238796} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_28415/1818135845.py", line 78, in objective
    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
  File "/tmp/ipykernel_28415/1289089649.py", line 65, in valid_fn_two
    for step, (images, labels) in pbar:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 628, in __next__
    data = self._next

KeyboardInterrupt: 

In [ ]:
fold=2
valid_df = df[df['fold']==fold].reset_index(drop=True)
valid_df = valid_df[valid_df['site_id']==2].reset_index(drop=True)
valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                num_workers=1, shuffle=False, drop_last=False)
set_seed(1)
out_file = 'swa_model_fold2_5.pth' 
iteration = [
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth',
    'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth',
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
#     a1 = 0.4360187712961733
#     a2 = 0.31005592868022136
#     a3 = 0.25392530002360536
#     a1 = 0.015006661988523864 
#     a2 = 0.12003546043452194 
#     a3 = 0.8649578775769542
#     a1 = 0.020317850755860567 
#     a2 = 0.1293785181217534 
#     a3 = 0.850303631122386
#     a1 = 0.12634002523631388
#     a2 = 0.8351954705276587
#     a3 = 0.03846450423602743
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = 1-a1-a2
    # a3 = trial.suggest_uniform('a3', 0.003, 1-a1-a2)
    # a1 = 0.4700450486328235 
    # a2 = 0.23862687145742947 
    # a3 = 0.2913280799097471
    state_dict = None
    for i in iteration:
        f = i
        # print(f)
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
#         elif i=='fold1/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth':
#             print("noobie", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
    print(a1, a2, a3)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = ModelOld(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold2_5.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    if best_metric>0.55:
        state = {'state_dict': model.state_dict()}
        path = f'site2_swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.3f}.pth'
        torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    return best_metric

study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=777))
study.optimize(func=objective, n_trials=200)
study.best_params

[I 2023-02-17 23:37:34,893] A new study created in memory with name: no-name-7294d8e0-af54-4852-bd01-b810915cd988


> SEEDING DONE
none:  fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4670_0.406.pth
hehe fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4746_0.314.pth
noob fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4469_0.454.pth
0.15198443381740748 0.25672863401172846 0.591286932170864



Val:   0%|          | 0/77 [00:00<?, ?it/s]
[W 2023-02-17 23:37:42,143] Trial 0 failed with parameters: {'a1': 0.15198443381740748, 'a2': 0.25672863401172846} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_28415/1352320558.py", line 78, in objective
    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
  File "/tmp/ipykernel_28415/1289089649.py", line 65, in valid_fn_two
    for step, (images, labels) in pbar:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 628, in __next__
    data = self._next_data()
  File "/home/tungnx/miniconda3/env

KeyboardInterrupt: 

# Fold 1

In [ ]:
# set_seed(1)
# out_file = 'swa_model_fold1_5.pth' 
# iteration = [
#     'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth',
    
#     'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth',
#     'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth',
#     'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth',
#     'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth',
#     # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_11_0.4211_0.242.pth',
#     # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4231_0.320.pth'
# ]

# criterion = nn.CrossEntropyLoss().to(CFG.device)
# best_metric = 0
# torch.cuda.empty_cache()
# def objective(trial):
#     a1 = trial.suggest_uniform('a1', 0.001, 0.99)
#     a2 = trial.suggest_uniform('a2', 0.0001, 1-a1-0.001)
#     a3 = trial.suggest_uniform('a3', 0.0001, 1-a1-a2-0.001)
#     a4 = trial.suggest_loguniform('a4', 0.0001, 1-a1-a2-a3-0.001)
#     a5 = 1-a1-a2-a3-a4
#     # a5 = trial.suggest_loguniform('a5', 0.0001, 1-a1-a2-a3-a4-0.001)
#     # a6 = trial.suggest_loguniform('a6', 0.0001, 1-a1-a2-a3-a4-a5-0.001)
#     # a7 = 1-a1-a2-a3-a4-a5-a6
#     state_dict = None
#     for i in iteration:
#         f = i
#         print(f)
#         f = torch.load(f, map_location=lambda storage, loc: storage)
#         if state_dict is None:
#             print("none: ", i)
#             state_dict = f['state_dict']
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = f['state_dict'][k]*a1
#         elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth': 
#             print("hehe", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
#         elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
#         elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
                
#         elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth':
#             print("noob", i)
#             key = list(f['state_dict'].keys())
#             for k in key:
#                 state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
                
#         # elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_11_0.4211_0.242.pth': 
#         #     print("hehe", i)
#         #     key = list(f['state_dict'].keys())
#         #     for k in key:
#         #         state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
                
#         # elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4231_0.320.pth': 
#         #     print("hehe", i)
#         #     key = list(f['state_dict'].keys())
#         #     for k in key:
#         #         state_dict[k] = state_dict[k] + a7*f['state_dict'][k]
#     print(a1, a2, a3, a4, a5)
#     # for k in key:
#     #     state_dict[k] = state_dict[k] / len(iteration)
#     print('')

#     # print(out_file)
#     torch.save({'state_dict': state_dict}, out_file)

#     model = ModelOld(model_name=CFG.model_name).to(CFG.device)
#     checkpoint = torch.load("swa_model_fold1_5.pth")
#     model.load_state_dict(checkpoint['state_dict'])
# #     model = nn.DataParallel(model)

#     loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
#     valid_preds = valid_preds[:, 1]
#     valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
#     valid_preds = np.array(valid_preds).flatten()
    
#     valid_df['raw_pred'] = valid_preds
#     LOGGER.info(f"Valid loss:{loss_valid:.4f}")
#     grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
#     grp_df['cancer'] = grp_df['cancer'].astype(np.int)
#     valid_labels_mean = grp_df['cancer'].values
#     valid_preds_mean = grp_df['raw_pred'].values
#     # print(valid_labels[:5], valid_preds_mean[:5])
#     val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
#     LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
#     best_metric_mean_at_epoch = 0
#     best_metric = 0
    
#     best_threshold_mean = 0
#     best_auc = 0
#     best_cf = None
#     for i in np.arange(0.001, 0.599, 0.001):
#         valid_argmax = (valid_preds_mean>i).astype(np.int32)
#         val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
#         val_acc = accuracy_score(valid_labels_mean, valid_argmax)
#         val_f1 = f1_score(valid_labels_mean, valid_argmax)
#         val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
#         cf = confusion_matrix(valid_labels_mean, valid_argmax)
#         if val_metric> best_metric:
#             best_metric = val_metric
#             # best_metric_mean_at_epoch = val_metric
#             best_threshold_mean = i
#             best_auc = val_auc
#             best_cf = cf
#     if best_metric>0.49:
#         state = {'state_dict': model.state_dict()}
#         path = f'swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.4f}.pth'
#         torch.save(state, path)
    
#     LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
#     LOGGER.info(f"Cf: {best_cf}")
#     return best_metric
# print("Find fold 1")
# study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=1))
# study.optimize(func=objective, n_trials=200)
# study.best_params

[I 2023-02-17 22:57:17,185] A new study created in memory with name: no-name-ee7c3426-82ad-490b-8656-a26f5656adda


> SEEDING DONE
Find fold 1
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
none:  fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
hehe fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
0.41343476265084567 0.42182495052152247 0.00011871632791617526 0.0009368038859990152 0.16368476661371667



Val: 100%|██████████| 170/170 [03:01<00:00,  1.07s/it, eval_loss=0.0882, gpu_mem=7.92 GB]
Valid loss:0.0882
Val metric mean prob: 0.2898
Best metric at: 0.4725 0.3580  0.7068
Cf: [[4625   37]
 [  59   43]]
[I 2023-02-17 23:00:27,018] Trial 0 finished with value: 0.47252747252747257 and parameters: {'a1': 0.41343476265084567, 'a2': 0.42182495052152247, 'a3': 0.00011871632791617526, 'a4': 0.0009368038859990152}. Best is trial 0 with value: 0.47252747252747257.


fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
none:  fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
hehe fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
0.1461415760181248 0.07884251454774104 0.14424974087974424 0.0020551672478268017 0.6287110013065631



Val: 100%|██████████| 170/170 [03:06<00:00,  1.10s/it, eval_loss=0.0871, gpu_mem=7.92 GB]
Valid loss:0.0871
Val metric mean prob: 0.2753
Best metric at: 0.4767 0.3410  0.7207
Cf: [[4617   45]
 [  56   46]]
[I 2023-02-17 23:03:42,574] Trial 1 finished with value: 0.4766839378238342 and parameters: {'a1': 0.1461415760181248, 'a2': 0.07884251454774104, 'a3': 0.14424974087974424, 'a4': 0.0020551672478268017}. Best is trial 1 with value: 0.4766839378238342.


...

fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
none:  fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
hehe fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
0.012330039789369474 0.5611299321580077 0.2983089691126598 0.11648516211586564 0.011745896824097365



Val: 100%|██████████| 170/170 [03:06<00:00,  1.10s/it, eval_loss=0.0796, gpu_mem=7.92 GB]
Valid loss:0.0796
Val metric mean prob: 0.2277
Best metric at: 0.4787 0.2530  0.7162
Cf: [[4621   41]
 [  57   45]]
[I 2023-02-17 23:32:03,942] Trial 10 finished with value: 0.4787234042553191 and parameters: {'a1': 0.012330039789369474, 'a2': 0.5611299321580077, 'a3': 0.2983089691126598, 'a4': 0.11648516211586564}. Best is trial 10 with value: 0.4787234042553191.


fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
none:  fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
hehe fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth
fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
noob fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth
0.010632516058786012 0.5205266514514182 0.32135025901555847 0.13959981515199907 0.007890758322238262



Val:  33%|███▎      | 56/170 [01:00<02:02,  1.08s/it, eval_loss=0.0771, gpu_mem=7.92 GB]
[W 2023-02-17 23:33:11,495] Trial 11 failed with parameters: {'a1': 0.010632516058786012, 'a2': 0.5205266514514182, 'a3': 0.32135025901555847, 'a4': 0.13959981515199907} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_28415/1275780627.py", line 83, in objective
    loss_valid, valid_preds = valid_fn_two(valid_loader, model, criterion, CFG.device)
  File "/tmp/ipykernel_28415/1289089649.py", line 65, in valid_fn_two
    for step, (images, labels) in pbar:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/tqdm/std.py", line 1195, in __iter__
    for obj in iterable:
  File "/home/tungnx/miniconda3/envs/zaloenv/lib/python3.10/site-packages/torch/utils/data/da

KeyboardInterrupt: 

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size = CFG.train_bs,
#                                   num_workers=1, shuffle=True, pin_memory=True, drop_last=True)
# for step, (images, labels) in tqdm(enumerate(train_loader)):
#     image0 = images[0, :, :, :]
#     image0 = torch.permute(image0, (1, 2, 0))
#     image0 = image0.cpu().numpy().astype(np.uint8)
#     plt.imshow(image0)
#     plt.show()
#     break

### 10folds b2v2 x1 summary:
### epoch 13 fold 0/10 epoch 2 vs 13 (0.673170477524555 0.3268295224754445) 0.5979 cv
### epoch 3,4,5,6,7,8 fold 1/10 needs retrain? because stop at epoch 4
### 2/3/2023 epoch 3, 5, 6, 8, fold 1/10 better CV
### epoch 7 fold 2/10
### no epoch fold 3/10
### epoch 7,8 fold 4/10 needs retrain because starts after fold 3
### epoch 6 fold 4 better CV
### epoch 5,8,11,13 fold 5/10
### epoch 6,7,9 fold 6/10
### epoch 7 fold 7/10 
### epoch ~ fold 8/10


### 10folds b2ns x1 summary
### epoch 5,7 fold 0/10 0.6304, 4, 7 fold 0/10 6263 
### epoch 2,7 fold 1/10
### epoch 4,7 fold 2/10

### 5 fold summaries:
### all good

In [ ]:
out_file = 'swa_model_fold1_10.pth' 
iteration = [
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4385_0.205.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4231_0.320.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4339_0.246.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_11_0.4211_0.242.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.4568_0.290.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4762_0.152.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_4_0.4151_0.352.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_5_0.4757_0.230.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_6_0.4520_0.128.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.4510_0.266.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_8_0.4403_0.415.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_9_0.4713_0.430.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_10_0.4569_0.259.pth',
    # 'fold0/tf_efficientnetv2_b2_fold_0_model_epoch_11_0.4387_0.436.pth'
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4000_0.122.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4585_0.236.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4149_0.131.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_6_0.4516_0.188.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_7_0.4557_0.241.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_8_0.4455_0.208.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_9_0.4681_0.319.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_10_0.4550_0.245.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_11_0.4500_0.373.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_12_0.4457_0.298.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_3_0.3867_0.302.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.3924_0.275.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_6_0.4030_0.339.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_5_0.3850_0.161.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4192_0.270.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_8_0.3913_0.362.pth'
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.4103_0.343.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_5_0.4041_0.141.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_6_0.4648_0.444.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4103_0.310.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_8_0.4471_0.371.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_10_0.4062_0.202.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4192_0.270.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_11_0.4309_0.199.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_12_0.4074_0.278.pth'
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_5_0.3889_0.407.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_4_0.4276_0.403.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_6_0.4000_0.586.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_7_0.3913_0.444.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_8_0.3916_0.483.pth'
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_3_0.4444_0.273.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_4_0.4533_0.439.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4416_0.357.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4400_0.230.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4118_0.398.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4000_0.442.pth'
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_3_0.4615_0.250.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_4_0.4272_0.261.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_5_0.4598_0.286.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_6_0.4118_0.425.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_7_0.4848_0.266.pth',
    # 'fold2/tf_efficientnetv2_b2_fold_2_model_epoch_10_0.4314_0.241.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_6_0.4675_0.377.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_8_0.4722_0.472.pth',
    # 'fold4/tf_efficientnetv2_b2_fold_4_model_epoch_7_0.4810_0.474.pth',
    # 'fold6/tf_efficientnetv2_b2_fold_6_model_epoch_6_0.5128_0.307.pth',
    # 'fold6/tf_efficientnetv2_b2_fold_6_model_epoch_7_0.5385_0.423.pth',
    # 'fold6/tf_efficientnetv2_b2_fold_6_model_epoch_9_0.5135_0.338.pth'
    # 'fold14/tf_efficientnetv2_b2_fold_1_model_epoch_3_0.4524_0.331.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4510_0.228.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4337_0.284.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4211_0.231.pth'
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_6_0.5349_0.350.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_10_0.5349_0.352.pth'
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_8_0.5316_0.367.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_9_0.5195_0.395.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_11_0.5287_0.326.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_12_0.5055_0.308.pth',
    # 'fold14/tf_efficientnetv2_b2_fold_4_model_epoch_13_0.5176_0.328.pth'
    # 'fold8/tf_efficientnetv2_b2_fold_8_model_epoch_5_0.4675_0.175.pth',
    # 'fold8/tf_efficientnetv2_b2_fold_8_model_epoch_13_0.4742_0.337.pth',
    # 'fold8/tf_efficientnetv2_b2_fold_8_model_epoch_8_0.4658_0.412.pth'
    # 'foldtest/tf_efficientnet_b2_ns_fold_0_model_epoch_4_0.5778_0.326.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_0_model_epoch_5_0.5783_0.535.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_0_model_epoch_7_0.5895_0.270.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_1_model_epoch_2_0.4706_0.370.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_1_model_epoch_7_0.4557_0.426.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_2_model_epoch_4_0.5234_0.334.pth',
    # 'foldtest/tf_efficientnet_b2_ns_fold_2_model_epoch_7_0.5000_0.234.pth',
    # 'foldonechannel/tf_efficientnetv2_b2_fold_0_model_epoch_4_0.5679_0.383.pth',
    # 'foldonechannel/tf_efficientnetv2_b2_fold_0_model_epoch_7_0.5636_0.153.pth',
    # 'foldonechannel/tf_efficientnetv2_b2_fold_0_model_epoch_8_0.5581_0.387.pth',
    # 'foldonechannel/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4837_0.455.pth',
    # 'foldonechannel/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4636_0.454.pth'
    'foldtest/tf_efficientnetv2_b2_fold_1_model_epoch_3_0.5055_0.360.pth',
    'foldtest/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4865_0.324.pth'
]
#46789101112 4824
state_dict = None
for i in iteration:
    f = i
    print(f)
    f = torch.load(f, map_location=lambda storage, loc: storage)
    if state_dict is None:
        state_dict = f['state_dict']
    else:
        key = list(f['state_dict'].keys())
        for k in key:
            state_dict[k] = state_dict[k] + f['state_dict'][k]

for k in key:
    state_dict[k] = state_dict[k] / len(iteration)
print('')

print(out_file)
torch.save({'state_dict': state_dict}, out_file)

foldtest/tf_efficientnetv2_b2_fold_1_model_epoch_3_0.5055_0.360.pth
foldtest/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4865_0.324.pth

swa_model_fold1_10.pth


In [ ]:
set_seed(1)
out_file = 'swa_model_fold1_5.pth' 
iteration = [
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4578_0.382.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4569_0.264.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4444_0.409.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4430_0.474.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4403_0.422.pth',
    'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth',
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
#     a1 = 0.036839841333967636 
#     a2 = 0.6490629183820655
#     a3 = 0.3140972402839668
#     a2 = 0.47142151346976024 
#     a3 = 0.3596277792186039
#     a1 = trial.suggest_uniform('a1', 0.01, 0.99)
#     a2 = 1-a1
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = trial.suggest_uniform('a3', 0.0009, 1-a1-a2-0.001)
    a4 = trial.suggest_loguniform('a4', 0.0009, 1-a1-a2-a3-0.001)
    a5 = trial.suggest_loguniform('a5', 0.0009, 1-a1-a2-a3-0.001)
    a6 = trial.suggest_loguniform('a6', 0.0009, 1-a1-a2-a3-0.001)
    a7 = 1-a1-a2-a3-a4-a5-a6
    # a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    # a2 = trial.suggest_uniform('a2', 0.0009, 1-a1)
    # a3 = trial.suggest_uniform('a3', 0.0007, 1-a1-a2)
    # a4 = trial.suggest_loguniform('a4', 0.0005, 1-a1-a2-a3)
    # a5 = trial.suggest_loguniform('a5', 0.00003, 1-a1-a2-a3-a4)
    # a6 = trial.suggest_loguniform('a6', 0.00009, 1-a1-a2-a3-a4-a5)
    # a7 = 1-a1-a2-a3-a4-a5-a6
    state_dict = None
    for i in iteration:
        f = i
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i== 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4569_0.264.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4444_0.409.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
        elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4432_0.319.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
                
        elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_7_0.4430_0.474.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
                
        elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_6_0.4403_0.422.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
                
        elif i=='fold1/tf_efficientnetv2_b2_fold_1_model_epoch_5_0.4393_0.278.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a7*f['state_dict'][k]
    print(a1, a2, a3, a4, a5, a6, a7)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = Model(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold1_5.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds, _ = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    state = {'state_dict': model.state_dict()}
    path = f'swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.4f}.pth'
    torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    return best_metric

study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=1))
study.optimize(func=objective, n_trials=30)
study.best_params

In [ ]:
def pfbeta_np(labels, preds, beta=1):
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.0
# fold=3
# valid_df = df[df['fold']==fold].reset_index(drop=True)
# valid_dataset = BreastDataset(valid_df, transforms=data_transforms['valid'])

valid_loader = DataLoader(valid_dataset, batch_size = CFG.valid_bs, 
                                num_workers=1, shuffle=False, drop_last=False)
set_seed(1)
out_file = 'swa_model_fold3_5.pth' 
iteration = [
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_8_0.4625_0.367.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_10_0.4766_0.251.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_12_0.4824_0.297.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_13_0.4771_0.241.pth',
    # 'fold1/tf_efficientnetv2_b2_fold_1_model_epoch_15_0.4878_0.242.pth'
    'foldsensemble/tf_efficientnetv2_b2_fold_3_model_epoch_6_0.4648_0.444.pth',
    'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4545_0.354.pth',
    'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_2_0.4528_0.320.pth',
    'foldsensemble/tf_efficientnetv2_b2_fold_3_model_epoch_8_0.4471_0.371.pth',
    'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.4379_0.392.pth',
    'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_3_0.4317_0.364.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_3_0.4224_0.288.pth',
    # 'foldsensemble/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4192_0.270.pth',
    # 'foldsensemble/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.4103_0.343.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_2_0.4528_0.320.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_3_0.4317_0.364.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.4311_0.361.pth',
    # 'fold3/tf_efficientnetv2_b2_fold_3_model_epoch_6_0.4304_0.332.pth'
]

criterion = nn.CrossEntropyLoss().to(CFG.device)
best_metric = 0
torch.cuda.empty_cache()
def objective(trial):
#     a2 = 0.12003546043452194 
#     a3 = 0.8649578775769542
#     a1 = 0.020317850755860567 
#     a2 = 0.1293785181217534 
#     a3 = 0.850303631122386
    # a1 = 0.2
    # a2 = 0.2    
    # a3 = 0.2
    # a4 = 0.2
    # a5 = 0.2
    # a1 = 1
    a1 = trial.suggest_uniform('a1', 0.001, 0.99)
    a2 = trial.suggest_uniform('a2', 0.0009, 1-a1-0.001)
    a3 = trial.suggest_loguniform('a3', 0.00009, 1-a1-a2-0.001)
    # a4 = 1-a1-a2-a3
    a4 = trial.suggest_loguniform('a4', 0.000009, 1-a1-a2-a3-0.001)
    # a5 = 1-a1-a2-a3-a4
    a5 = trial.suggest_loguniform('a5', 0.0000009, 1-a1-a2-a3-a4-0.001)
    a6 = 1-a1-a2-a3-a4-a5
    # a4 = 1-a1-a2-a3
    # a1 = 0.4700450486328235 
    # a2 = 0.23862687145742947 
    # a3 = 0.2913280799097471
    state_dict = None
    for i in iteration:
        f = i
        # print(f)
        f = torch.load(f, map_location=lambda storage, loc: storage)
        if state_dict is None:
            print("none: ", i)
            state_dict = f['state_dict']
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = f['state_dict'][k]*a1
        elif i=='fold3/tf_efficientnetv2_b2_fold_3_model_epoch_7_0.4545_0.354.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a2*f['state_dict'][k]
        elif i=='fold3/tf_efficientnetv2_b2_fold_3_model_epoch_2_0.4528_0.320.pth': 
            print("hehe", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a3*f['state_dict'][k]
        elif i=='foldsensemble/tf_efficientnetv2_b2_fold_3_model_epoch_8_0.4471_0.371.pth':
            print("noob", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a4*f['state_dict'][k]
        elif i=='fold3/tf_efficientnetv2_b2_fold_3_model_epoch_4_0.4379_0.392.pth':
            print("noobie", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a5*f['state_dict'][k]
        elif i=='fold3/tf_efficientnetv2_b2_fold_3_model_epoch_3_0.4317_0.364.pth':
            print("noobie", i)
            key = list(f['state_dict'].keys())
            for k in key:
                state_dict[k] = state_dict[k] + a6*f['state_dict'][k]
    print(a1, a2, a3, a4, a5, a6)
    # for k in key:
    #     state_dict[k] = state_dict[k] / len(iteration)
    print('')

    # print(out_file)
    torch.save({'state_dict': state_dict}, out_file)

    model = Model(model_name=CFG.model_name).to(CFG.device)
    checkpoint = torch.load("swa_model_fold3_5.pth")
    model.load_state_dict(checkpoint['state_dict'])
#     model = nn.DataParallel(model)

    loss_valid, valid_preds, _ = valid_fn_two(valid_loader, model, criterion, CFG.device)
    valid_preds = valid_preds[:, 1]
    valid_df['prediction_id'] = valid_df['patient_id'].astype(str) + '_' + valid_df['laterality'].astype(str)
    valid_preds = np.array(valid_preds).flatten()
    
    valid_df['raw_pred'] = valid_preds
    LOGGER.info(f"Valid loss:{loss_valid:.4f}")
    grp_df = valid_df.groupby('prediction_id')['raw_pred', 'cancer'].mean()
    grp_df['cancer'] = grp_df['cancer'].astype(np.int)
    valid_labels_mean = grp_df['cancer'].values
    valid_preds_mean = grp_df['raw_pred'].values
    # print(valid_labels[:5], valid_preds_mean[:5])
    val_metric_mean = pfbeta(valid_labels_mean, valid_preds_mean)
    LOGGER.info(f"Val metric mean prob: {val_metric_mean:.4f}")
    best_metric_mean_at_epoch = 0
    best_metric = 0
    
    best_threshold_mean = 0
    best_auc = 0
    best_cf = None
    for i in np.arange(0.001, 0.599, 0.001):
        valid_argmax = (valid_preds_mean>i).astype(np.int32)
        val_metric = pfbeta_np(valid_labels_mean, valid_argmax)
        val_acc = accuracy_score(valid_labels_mean, valid_argmax)
        val_f1 = f1_score(valid_labels_mean, valid_argmax)
        val_auc = roc_auc_score(valid_labels_mean, valid_argmax)
        cf = confusion_matrix(valid_labels_mean, valid_argmax)
        if val_metric> best_metric:
            best_metric = val_metric
            # best_metric_mean_at_epoch = val_metric
            best_threshold_mean = i
            best_auc = val_auc
            best_cf = cf
    if best_metric>0.51:
        state = {'state_dict': model.state_dict()}
        path = f'swa_{CFG.model_name}_fold_{fold}_model_{best_metric:.4f}_{best_threshold_mean:.3f}.pth'
        torch.save(state, path)
    
    LOGGER.info(f"Best metric at: {best_metric:.4f} {best_threshold_mean:.4f}  {best_auc:.4f}")
    LOGGER.info(f"Cf: {best_cf}")
    torch.cuda.empty_cache()
    return best_metric

study = optuna.create_study(direction='maximize', sampler = TPESampler(seed=666))
study.optimize(func=objective, n_trials=200)
study.best_params